# Denmark monthly field-by-field oil production data

Link: https://ens.dk/sites/ens.dk/files/OlieGas/HTM_GIF/mp201801ofu.htm
      https://ens.dk/sites/ens.dk/files/OlieGas/HTM_GIF/mp201907ofu.htm
      


In [2]:
%cd ..

C:\Users\ROSA_L\PycharmProjects\scraper


In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd

df = pd.read_html('https://ens.dk/sites/ens.dk/files/OlieGas/HTM_GIF/mp201801ofu.htm')[0]
df[0:30]

,0,1,2,3,4,5
0,"Danish Production of Oil, Gas and Water for Ja...",NaN,NaN,NaN,NaN,NaN
1,"Oil, 1000 barrels",January 2018,Oct17-Dec17,Oct16-Dec16,This year,NaN
2,Field,Monthly,Daily Avg.,Daily Avg.,Daily Avg.,NaN
3,Dan,5980,193,212,211,5980
4,Gorm,1511,49,60,63,1511
5,Skjold,2172,70,96,107,2172
6,Tyra,2091,67,68,68,2091
7,Rolf,211,07,08,12,211
8,Kraka,691,22,19,20,691
9,Dagmar,-,-,-,-,-


In [4]:
#breakIndex = df.loc[df[0]df.isnull().values.any()].index
# all columns null is a break between tables
break_indexes = df[df.isnull().all(axis=1)].index
break_indexes[0]

24

In [5]:
oil_table_start = 3
oil_table_end = break_indexes[0]
table = df[oil_table_start:oil_table_end]
table.columns = df.loc[2].values
table = table[['Field', 'Monthly']]
table['Monthly'].replace('-', 0, inplace=True)

In [6]:
table

,Field,Monthly
3,Dan,5980
4,Gorm,1511
5,Skjold,2172
6,Tyra,2091
7,Rolf,211
8,Kraka,691
9,Dagmar,0
10,Regnar,0
11,Valdemar,3614
12,Roar,569


In [59]:
from datetime import date
int(date.today().replace(day=1).strftime('%Y%m')) - 2

201907

In [7]:
table.rename(columns={'Field': 'entity',
                       'Monthly': 'value'})

,entity,value
3,Dan,5980
4,Gorm,1511
5,Skjold,2172
6,Tyra,2091
7,Rolf,211
8,Kraka,691
9,Dagmar,0
10,Regnar,0
11,Valdemar,3614
12,Roar,569


In [36]:
from datetime import datetime
header = df.iloc[0,0].split(' ')
period = ' '.join(header[-2:])
dt = datetime.strptime(period, '%B %Y').date()
dt.month, dt.year


(1, 2018)

In [64]:
#table.drop(columns=['entity'], inplace=True)
table.entity = table.apply(lambda x: f'DK_ENS_{x["Field"].upper()}', axis=1)
table

,Field,Monthly
3,Dan,5980
4,Gorm,1511
5,Skjold,2172
6,Tyra,2091
7,Rolf,211
8,Kraka,691
9,Dagmar,0
10,Regnar,0
11,Valdemar,3614
12,Roar,569


In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

In [4]:
from scraper.core.job import Job
from scraper.jobs.dk_ens.job import DenmarkOilProdJob
from scraper.settings import API_END_POINT, FILE_STORE_PATH
import sys, traceback
try:
    dk = DenmarkOilProdJob()
    dk.run()

except:
    print("Exception in user code:")
    print('-'*60)
    traceback.print_exc(file=sys.stdout)
    print('-'*60)

2020-03-03 11:08:52,009 - scraper.jobs.dk_ens.job - DEBUG - Generating sources...
2020-03-03 11:08:52,013 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): vipenta:8000
2020-03-03 11:08:52,499 - urllib3.connectionpool - DEBUG - http://vipenta:8000 "GET /dimension/source HTTP/1.1" 200 707658
2020-03-03 11:08:52,518 - scraper.core.job - DEBUG - download: True, parallel download: True
2020-03-03 11:08:52,519 - scraper.jobs.dk_ens.job - DEBUG - calling super(cls, cls).download_source(source)
2020-03-03 11:08:52,522 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): ens.dk:443
2020-03-03 11:08:52,779 - urllib3.connectionpool - DEBUG - https://ens.dk:443 "GET /sites/ens.dk/files/OlieGas/HTM_GIF/mp202002ofu.htm HTTP/1.1" 404 6131
2020-03-03 11:08:52,782 - scraper.core.job - ERROR - Issue Download Error 
 url: https://ens.dk/sites/ens.dk/files/OlieGas/HTM_GIF/mp202002ofu.htm 
 status_code: 404 in source with code dk_ens_crudeoil_202002 and content b'<!DOCT

In [29]:
[vars(source) for source in dk.sources]

[{'code': 'dk_ens_crudeoil_201910',
  'url': 'https://ens.dk/sites/ens.dk/files/OlieGas/HTM_GIF/mp201910ofu.htm',
  'path': 'dk_ens_crudeoil_201910.html',
  'long_name': 'DENMARK DK_ENS Oil Monthly Production by Field - 201910'}]

In [27]:
from datetime import date
from dateutil.relativedelta import relativedelta
import pandas as pd

PUBLICATION_DELAY=2
current_period = date.today().replace(day=1) - relativedelta(months=PUBLICATION_DELAY)
start_date = current_period
end_date = current_period
print(f"s: P{start_date} e: {end_date}")
[x for x in pd.date_range(start_date, end_date, freq='MS')]

s: P2019-09-01 e: 2019-09-01


[Timestamp('2019-09-01 00:00:00', freq='MS')]

In [24]:
import pandas as pd
pd.read_html('C:\\Users\\ROSA_L\\PycharmProjects\\scrapper\\filestore\\mp201106ofu.htm')[0]

ValueError: No tables found

In [45]:
import pandas as pd

df_excel = pd.read_excel('G:\\OMRsupply\\Input files\\Denmark.xlsm', sheet_name='Jun2011')
df_excel.replace('I alt', 'Total')
df_excel.append(pd.Series(), ignore_index=True)
df_excel.to_html('C:\\Users\\ROSA_L\\Downloads\\DK\\dk_ens_crudeoil_201106.html', index_names=False)

In [55]:
df_excel = pd.read_html('C:\\Users\\ROSA_L\\Downloads\\DK\\dk_ens_crudeoil_201106.html')
df_excel

[                                                   0        1              2   \
 0   Danish Production of Oil, Gas and Water for Ju...      NaN            NaN   
 1                                   Oil, 1000 barrels   Jun-11  mar 11-maj 11   
 2                                                 NaN      NaN            NaN   
 3                                               Field  Monthly     Daily Avg.   
 4                                                 Dan   1.3988            466   
 5                                                Gorm     3288            110   
 6                                              Skjold     3824            127   
 7                                                Tyra     3941            131   
 8                                                Rolf        -              -   
 9                                               Kraka     1134             38   
 10                                             Dagmar        -              -   
 11             

In [17]:
# convert historical data from denmark to html so we can read it with our process
import win32com.client as win32
from win32com.client import constants as c
import datetime
import pandas as pd

start=datetime.datetime(2012, 8, 1)
end=datetime.datetime(2013, 9, 1)
date_range = pd.date_range(start, end, freq='BM')
excel = win32.gencache.EnsureDispatch('Excel.Application') 
wb = excel.Workbooks.Open(r'G:\\OMRsupply\\Input files\\Denmark.xlsm') 
excel.Visible = True

for i, d in enumerate(date_range, 1):
    try:
        sheet = d.strftime("%b%Y")
        sheet_range = '$A$1:$L$25' if d > datetime.datetime(2015, 5, 1) else '$A$1:$F$24'
        ws = wb.Worksheets(sheet)
        dt = d.strftime("%Y%m")
        filename = f'C:\\Users\\ROSA_L\\Downloads\\DK\\dk_ens_crudeoil_{dt}.html'
        print(f'i: {i} d: {d} f: {filename} range {sheet_range}')
        x = wb.PublishObjects.Add(SourceType=c.xlSourceRange,Filename=filename,Sheet=sheet,Source=sheet_range,HtmlType=c.xlHtmlStatic, DivID='xxx1')
        #x.PublishObjects(1).Publish(True)
        x.Publish(True)
        print(f'period {d} saved')
    except Exception as e:
        print(e)
print("Fini")

i: 1 d: 2012-08-31 00:00:00 f: C:\Users\ROSA_L\Downloads\DK\dk_ens_crudeoil_201208.html range $A$1:$F$24
period 2012-08-31 00:00:00 saved
i: 2 d: 2012-09-28 00:00:00 f: C:\Users\ROSA_L\Downloads\DK\dk_ens_crudeoil_201209.html range $A$1:$F$24
period 2012-09-28 00:00:00 saved
i: 3 d: 2012-10-31 00:00:00 f: C:\Users\ROSA_L\Downloads\DK\dk_ens_crudeoil_201210.html range $A$1:$F$24
period 2012-10-31 00:00:00 saved
i: 4 d: 2012-11-30 00:00:00 f: C:\Users\ROSA_L\Downloads\DK\dk_ens_crudeoil_201211.html range $A$1:$F$24
period 2012-11-30 00:00:00 saved
i: 5 d: 2012-12-31 00:00:00 f: C:\Users\ROSA_L\Downloads\DK\dk_ens_crudeoil_201212.html range $A$1:$F$24
period 2012-12-31 00:00:00 saved
i: 6 d: 2013-01-31 00:00:00 f: C:\Users\ROSA_L\Downloads\DK\dk_ens_crudeoil_201301.html range $A$1:$F$24
period 2013-01-31 00:00:00 saved
i: 7 d: 2013-02-28 00:00:00 f: C:\Users\ROSA_L\Downloads\DK\dk_ens_crudeoil_201302.html range $A$1:$F$24
period 2013-02-28 00:00:00 saved
i: 8 d: 2013-03-29 00:00:00 f: C:\

In [90]:
from datetime import date
source = "dk_ens_crudeoil_200801" 
period = source.split('_')[-1]
date_period =date(int(period[0:4]), int(period[-2:]), 1)
print(f"date: {date_period}")

date: 2008-01-01


In [16]:
import pandas as pd

THOUSANDS_SEP = '.'
DECIMAL_SEP = ','

df = pd.read_html("file:///C:/Users/ROSA_L/PycharmProjects/scrapper/filestore/dk_ens_crudeoil_201611.html",  thousands=THOUSANDS_SEP, decimal=DECIMAL_SEP)
df = df[0]
field_text_indexes = df[df.apply(lambda r: r.str.contains('Field', case=True).any(), axis=1)].index
oil_table_start = field_text_indexes[0]
table = df[oil_table_start:]
break_indexes = table[df.apply(lambda r: r.str.contains('Total', case=True).any(), axis=1)].index
df.columns = df.loc[oil_table_start].values
oil_table_end = break_indexes[0]
table = df[oil_table_start+1:oil_table_end]
table = table[['Field', 'Monthly']]
table['Monthly'].replace('-', "0", inplace=True)
table['value'] = table['Monthly'].astype(float)
table
df

c:\users\rosa_l\pycharmprojects\scrapper\venv\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()


,Field,Monthly,Daily Avg.,Daily Avg.,Daily Avg.,nan,nan,nan,nan,nan,nan,nan
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Oil, 1000 barrels",Nov-16,aug 16-okt 16,aug 15-okt 15,This Year,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Field,Monthly,Daily Avg.,Daily Avg.,Daily Avg.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Dan,6311,21,219,272,7.75280,NaN,NaN,NaN,NaN,NaN,NaN
6,Gorm,198,66,63,8,2.07730,NaN,NaN,NaN,NaN,NaN,NaN
7,Skjold,3187,106,88,84,2.92040,NaN,NaN,NaN,NaN,NaN,NaN
8,Tyra,2152,72,73,85,2.51970,NaN,NaN,NaN,NaN,NaN,NaN
9,Rolf,39,13,1,3,4387,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
import datetime

start=datetime.datetime(2011, 1, 1)
end=datetime.datetime(2017, 12, 1)
date_range = pd.date_range(start, end, freq='BM')
for d in date_range:
    print(d.strftime("%b%Y"))

Jan2011
Feb2011
Mar2011
Apr2011
May2011
Jun2011
Jul2011
Aug2011
Sep2011
Oct2011
Nov2011
Dec2011
Jan2012
Feb2012
Mar2012
Apr2012
May2012
Jun2012
Jul2012
Aug2012
Sep2012
Oct2012
Nov2012
Dec2012
Jan2013
Feb2013
Mar2013
Apr2013
May2013
Jun2013
Jul2013
Aug2013
Sep2013
Oct2013
Nov2013
Dec2013
Jan2014
Feb2014
Mar2014
Apr2014
May2014
Jun2014
Jul2014
Aug2014
Sep2014
Oct2014
Nov2014
Dec2014
Jan2015
Feb2015
Mar2015
Apr2015
May2015
Jun2015
Jul2015
Aug2015
Sep2015
Oct2015
Nov2015
Dec2015
Jan2016
Feb2016
Mar2016
Apr2016
May2016
Jun2016
Jul2016
Aug2016
Sep2016
Oct2016
Nov2016
Dec2016
Jan2017
Feb2017
Mar2017
Apr2017
May2017
Jun2017
Jul2017
Aug2017
Sep2017
Oct2017
Nov2017


In [39]:
import requests
r = requests.get('https://ens.dk/sites/ens.dk/files/OlieGas/HTM_GIF/mp201909ofu.htm', verify=False)
print(r.status_code)


404


c:\users\rosa_l\pycharmprojects\scrapper\venv\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [8]:
e = OSError("Test")
logger.warning(e)
print(e)

Test


In [168]:
[vars(src) for src in dk.sources]

[{'code': 'dk_ens_crudeoil_201909',
  'url': 'https://ens.dk/sites/ens.dk/files/OlieGas/HTM_GIF/mp201909ofu.htm',
  'path': 'dk_ens_crudeoil_201909.html',
  'long_name': 'DENMARK DK_ENS Oil Monthly Production by Field - 201909'}]

In [25]:
from itertools import chain, product
lista = chain(product(reversed(range(2019, 2020)), reversed(range(1, 8))), product(reversed(range(2017, 2019)), reversed(range(1, 13))))
[l for l in lista]

[(2019, 7),
 (2019, 6),
 (2019, 5),
 (2019, 4),
 (2019, 3),
 (2019, 2),
 (2019, 1),
 (2018, 12),
 (2018, 11),
 (2018, 10),
 (2018, 9),
 (2018, 8),
 (2018, 7),
 (2018, 6),
 (2018, 5),
 (2018, 4),
 (2018, 3),
 (2018, 2),
 (2018, 1),
 (2017, 12),
 (2017, 11),
 (2017, 10),
 (2017, 9),
 (2017, 8),
 (2017, 7),
 (2017, 6),
 (2017, 5),
 (2017, 4),
 (2017, 3),
 (2017, 2),
 (2017, 1)]

In [50]:
dk.download_and_get_checksum(True)

## Fixing get_source() to get existing months
Sometimes, we silently lose DK months that don't get processed.
It happens when they publish their numbers with a delay greater than expected (one month): in this case, our process shift to the following month and lose the latest published data.

To fix it, we need to get the available data in https://ens.dk/en/our-services/oil-and-gas-related-data/monthly-and-yearly-production


In [22]:
import requests
from bs4 import BeautifulSoup

response = requests.get('https://ens.dk/en/our-services/oil-and-gas-related-data/monthly-and-yearly-production')

if not response.ok:
    response.raise_for_status()
    
bf = BeautifulSoup(response.content)

In [79]:
import re
from datetime import datetime

#bf.find('table').find('tbody').find_all('tr')[0].find_all('td')[1]

source_list = bf.find_all('a', href=re.compile('ofu.htm'))

formatted_list = ({'period': datetime.strptime(s.text, '%B %Y').date(), 'url': s['href']} for s in source_list)

latest_source = next(iter(sorted(formatted_list, key=lambda x: x['period'], reverse=True)))
latest_source['period']

datetime.date(2020, 11, 1)

In [31]:
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta
# First period with data in history
START_PERIOD = date(2011, 6, 1)
# First period available online
START_DOWNLOAD = date(2018, 1, 1)

last_offline = START_DOWNLOAD + relativedelta(months=-1)

l = reversed(pd.date_range(START_PERIOD, last_offline, freq='MS'))

In [32]:
for i in l:
    mydate = i.date()
    print(type(mydate))
    print(i)
    

<class 'datetime.date'>
2017-12-01 00:00:00
<class 'datetime.date'>
2017-11-01 00:00:00
<class 'datetime.date'>
2017-10-01 00:00:00
<class 'datetime.date'>
2017-09-01 00:00:00
<class 'datetime.date'>
2017-08-01 00:00:00
<class 'datetime.date'>
2017-07-01 00:00:00
<class 'datetime.date'>
2017-06-01 00:00:00
<class 'datetime.date'>
2017-05-01 00:00:00
<class 'datetime.date'>
2017-04-01 00:00:00
<class 'datetime.date'>
2017-03-01 00:00:00
<class 'datetime.date'>
2017-02-01 00:00:00
<class 'datetime.date'>
2017-01-01 00:00:00
<class 'datetime.date'>
2016-12-01 00:00:00
<class 'datetime.date'>
2016-11-01 00:00:00
<class 'datetime.date'>
2016-10-01 00:00:00
<class 'datetime.date'>
2016-09-01 00:00:00
<class 'datetime.date'>
2016-08-01 00:00:00
<class 'datetime.date'>
2016-07-01 00:00:00
<class 'datetime.date'>
2016-06-01 00:00:00
<class 'datetime.date'>
2016-05-01 00:00:00
<class 'datetime.date'>
2016-04-01 00:00:00
<class 'datetime.date'>
2016-03-01 00:00:00
<class 'datetime.date'>
2016-02-

In [10]:
from scraper.core import factory

job = factory.get_scraper_job('dk_ens', 'dk_oil_prod', full_load=True)

2021-02-24 15:41:35,446 - scraper.core.factory - DEBUG - Loading module scraper.jobs.dk_ens.dk_oil_prod
2021-02-24 15:41:35,447 - scraper.core.factory - DEBUG - Getting class DkOilProdJob


In [8]:
job.get_sources()

2021-02-24 15:39:29,613 - scraper.jobs.dk_ens.dk_oil_prod - DEBUG - Generating sources...
2021-02-24 15:39:29,615 - scraper.jobs.dk_ens.dk_oil_prod - INFO - Getting list of online sources from https://ens.dk/en/our-services/oil-and-gas-related-data/monthly-and-yearly-production
2021-02-24 15:39:29,618 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): ens.dk:443
2021-02-24 15:39:29,967 - urllib3.connectionpool - DEBUG - https://ens.dk:443 "GET /en/our-services/oil-and-gas-related-data/monthly-and-yearly-production HTTP/1.1" 200 None
2021-02-24 15:39:30,058 - scraper.jobs.dk_ens.dk_oil_prod - DEBUG - Parsing online sources (anchors with href containing ofu.htm)
2021-02-24 15:39:30,101 - scraper.jobs.dk_ens.dk_oil_prod - INFO - 37 online sources found in the website.
2021-02-24 15:39:30,103 - scraper.jobs.dk_ens.dk_oil_prod - INFO - Current period: 2021-01-01
2021-02-24 15:39:30,103 - scraper.jobs.dk_ens.dk_oil_prod - INFO - Loading all online periods plus those availa

In [9]:
[vars(s) for s in job.sources]

[{'code': 'dk_ens_crudeoil_202101',
  'url': '/sites/ens.dk/files/OlieGas/mp202101ofu.htm',
  'path': 'dk_ens_crudeoil_202101.html',
  'long_name': 'DENMARK DK_ENS Oil Monthly Production by Field - 202101'},
 {'code': 'dk_ens_crudeoil_202012',
  'url': '/sites/ens.dk/files/OlieGas/HTM_GIF/mp202012ofu.htm',
  'path': 'dk_ens_crudeoil_202012.html',
  'long_name': 'DENMARK DK_ENS Oil Monthly Production by Field - 202012'},
 {'code': 'dk_ens_crudeoil_202011',
  'url': '/sites/ens.dk/files/OlieGas/HTM_GIF/mp202011ofu.htm',
  'path': 'dk_ens_crudeoil_202011.html',
  'long_name': 'DENMARK DK_ENS Oil Monthly Production by Field - 202011'},
 {'code': 'dk_ens_crudeoil_202010',
  'url': '/sites/ens.dk/files/OlieGas/HTM_GIF/mp202010ofu.htm',
  'path': 'dk_ens_crudeoil_202010.html',
  'long_name': 'DENMARK DK_ENS Oil Monthly Production by Field - 202010'},
 {'code': 'dk_ens_crudeoil_202009',
  'url': '/sites/ens.dk/files/OlieGas/HTM_GIF/mp202009ofu.htm',
  'path': 'dk_ens_crudeoil_202009.html',
  '

In [15]:
from scraper.core import factory

job = factory.get_scraper_job('dk_ens', 'dk_oil_prod')
job.run()

2021-02-24 15:53:55,490 - scraper.core.factory - DEBUG - Loading module scraper.jobs.dk_ens.dk_oil_prod
2021-02-24 15:53:55,491 - scraper.core.factory - DEBUG - Getting class DkOilProdJob
2021-02-24 15:53:55,492 - scraper.jobs.dk_ens.dk_oil_prod - DEBUG - Generating sources...
2021-02-24 15:53:55,494 - scraper.jobs.dk_ens.dk_oil_prod - INFO - Getting list of online sources from https://ens.dk/en/our-services/oil-and-gas-related-data/monthly-and-yearly-production
2021-02-24 15:53:55,498 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): ens.dk:443
2021-02-24 15:53:56,037 - urllib3.connectionpool - DEBUG - https://ens.dk:443 "GET /en/our-services/oil-and-gas-related-data/monthly-and-yearly-production HTTP/1.1" 200 None
2021-02-24 15:53:56,110 - scraper.jobs.dk_ens.dk_oil_prod - DEBUG - Parsing online sources (anchors with href containing ofu.htm)
2021-02-24 15:53:56,143 - scraper.jobs.dk_ens.dk_oil_prod - INFO - 37 online sources found in the website.
2021-02-24 15:53:

C:\Users\ROSA_L\PycharmProjects\scraper\scraper\jobs\dk_ens\dk_oil_prod.py:234: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  break_indexes = table[df.apply(lambda r: r.str.contains('Total', case=True).any(), axis=1)].index


2021-02-24 15:54:12,997 - urllib3.connectionpool - DEBUG - http://vipenta:8000 "GET /dimension/entity HTTP/1.1" 200 17233190
2021-02-24 15:54:36,620 - scraper.core.job - DEBUG - self.dynamic_dim['entity'] size before: 20
2021-02-24 15:54:36,621 - scraper.core.job - DEBUG - self.dynamic_dim['entity'] size after: 0
2021-02-24 15:54:36,644 - scraper.jobs.dk_ens.dk_oil_prod - DEBUG - Number of rows transformed: 20
2021-02-24 15:54:36,646 - scraper.core.job - DEBUG - Running insert_new_dynamic_dim(): 3 items
2021-02-24 15:54:36,647 - scraper.core.job - DEBUG - Processing source: size 0
2021-02-24 15:54:36,647 - scraper.core.job - DEBUG - Processing provider: size 0
2021-02-24 15:54:36,648 - scraper.core.job - DEBUG - Processing entity: size 0
2021-02-24 15:54:36,648 - scraper.core.utils - INFO - insert_new_dynamic_dim: 2.000570297241211 ms
2021-02-24 15:54:36,651 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): vipenta:8000
2021-02-24 15:54:38,208 - urllib3.connectionpoo